In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from admm.agents import GlobalConsensus, EventGlobalConsensus, EventGlobalConsensusTorch
from admm.models import NN, Dummy
from admm.utils import add_params, scale_params, subtract_params, average_params, sum_params
import torch
from tqdm import tqdm

%matplotlib inline

### Experimenting with SGD and adding parameters

In [2]:
def get_copy(params):
    copy = [torch.zeros(param.shape).copy_(param) for param in params]
    return copy

In [3]:
model1 = NN(2,2,1)

copy = get_copy(model1.parameters())

for param in copy:
    param.data = 2*torch.ones(param.shape)
for param in copy:
    print(param)

print('\n----- model 1 -----')
print('before training')
for param in model1.parameters():
    print(param)

# experiment minimizing l(x) = |x - 2|_2
opt = torch.optim.Adam(model1.parameters(), lr=0.01)
loss = torch.Tensor([np.inf])
prev_loss = torch.Tensor([0])
while np.abs(prev_loss.item() - loss.item()) >= 1e-4:
    prev_loss = loss
    opt.zero_grad()
    loss = 0
    for param, copied in zip(model1.parameters(), copy):
        loss += torch.norm(param-copied.data)
    loss.backward()
    opt.step()

print('\nafter training')
for param in model1.parameters():
    print(param)
print('\ncopied')
for param in copy:
    print(param)

tensor([[2., 2.],
        [2., 2.]], grad_fn=<CopyBackwards>)
tensor([2., 2.], grad_fn=<CopyBackwards>)
tensor([[2., 2.]], grad_fn=<CopyBackwards>)
tensor([2.], grad_fn=<CopyBackwards>)

----- model 1 -----
before training
Parameter containing:
tensor([[-0.2087, -0.4223],
        [ 0.4754, -0.4016]], requires_grad=True)
Parameter containing:
tensor([-0.3183, -0.5173], requires_grad=True)
Parameter containing:
tensor([[-0.6684, -0.4672]], requires_grad=True)
Parameter containing:
tensor([0.1109], requires_grad=True)

after training
Parameter containing:
tensor([[1.9995, 1.9998],
        [2.0004, 1.9997]], requires_grad=True)
Parameter containing:
tensor([2.0009, 2.0014], requires_grad=True)
Parameter containing:
tensor([[1.9951, 2.0026]], requires_grad=True)
Parameter containing:
tensor([2.0010], requires_grad=True)

copied
tensor([[2., 2.],
        [2., 2.]], grad_fn=<CopyBackwards>)
tensor([2., 2.], grad_fn=<CopyBackwards>)
tensor([[2., 2.]], grad_fn=<CopyBackwards>)
tensor([2.], grad

### Event-Based ADMM with Torch

In [4]:
# Initial lambdas must sum to 0!
delta = 0.01
rho = 1
N = 4

agents = [
    EventGlobalConsensusTorch(
        N=N, 
        rho=rho, 
        model=NN(2,3,1),
        delta=delta
    ) 
    for _ in range(N)
]

for agent in agents:
    agent.primal_avg = average_params([agent.model.parameters() for agent in agents])
    for param in agent.primal_avg:
        print(param)

for agent in agents:
    for param in agent.primal_avg:
        print(param)

print('\nbefore training')
for agent in agents:
    for param in agent.model.parameters(): print(param)

tensor([[-0.2295,  0.1047],
        [ 0.0283,  0.1310],
        [-0.3201,  0.1946]], grad_fn=<DivBackward0>)
tensor([-0.2881,  0.0984,  0.2138], grad_fn=<DivBackward0>)
tensor([[-0.0321,  0.0383, -0.1697]], grad_fn=<DivBackward0>)
tensor([0.0971], grad_fn=<DivBackward0>)
tensor([[-0.2295,  0.1047],
        [ 0.0283,  0.1310],
        [-0.3201,  0.1946]], grad_fn=<DivBackward0>)
tensor([-0.2881,  0.0984,  0.2138], grad_fn=<DivBackward0>)
tensor([[-0.0321,  0.0383, -0.1697]], grad_fn=<DivBackward0>)
tensor([0.0971], grad_fn=<DivBackward0>)
tensor([[-0.2295,  0.1047],
        [ 0.0283,  0.1310],
        [-0.3201,  0.1946]], grad_fn=<DivBackward0>)
tensor([-0.2881,  0.0984,  0.2138], grad_fn=<DivBackward0>)
tensor([[-0.0321,  0.0383, -0.1697]], grad_fn=<DivBackward0>)
tensor([0.0971], grad_fn=<DivBackward0>)
tensor([[-0.2295,  0.1047],
        [ 0.0283,  0.1310],
        [-0.3201,  0.1946]], grad_fn=<DivBackward0>)
tensor([-0.2881,  0.0984,  0.2138], grad_fn=<DivBackward0>)
tensor([[-0.032

### Run simulation

In [5]:
comm = 0

t_max = 200
for t in tqdm(range(t_max)):
    
    # Primal Update
    for agent in agents:
        agent.primal_update()

    # Residual update in the case of communication
    C = []
    for agent in agents:
        if agent.broadcast: 
            comm += 1
            C.append(agent.residual)
    if C:
        # If communicaiton set isn't empty
        residuals = [x for x in sum_params(C)]
        for agent in agents:
            add_params(agent.primal_avg, residuals)

    # Dual update
    for agent in agents:
        agent.dual_update()

load = comm/(t_max*len(agents))
print(f'Communication load = {load}')

print('\nParams')
for agent in agents:
    for param in agent.model.parameters():
        print(param)

print('\nAverage')
for agent in agents:
    for param in agent.primal_avg:
        print(param)

100%|██████████| 200/200 [00:13<00:00, 15.08it/s]

Communication load = 0.0825

Params
Parameter containing:
tensor([[2.0007, 1.9997],
        [2.0003, 2.0000],
        [1.9987, 2.0001]], requires_grad=True)
Parameter containing:
tensor([1.9992, 2.0009, 2.0010], requires_grad=True)
Parameter containing:
tensor([[2.0001, 2.0002, 1.9998]], requires_grad=True)
Parameter containing:
tensor([2.0008], requires_grad=True)
Parameter containing:
tensor([[1.9995, 2.0006],
        [1.9994, 1.9994],
        [2.0005, 2.0005]], requires_grad=True)
Parameter containing:
tensor([2.0008, 2.0004, 2.0008], requires_grad=True)
Parameter containing:
tensor([[1.9995, 2.0000, 2.0001]], requires_grad=True)
Parameter containing:
tensor([1.9994], requires_grad=True)
Parameter containing:
tensor([[1.9993, 2.0006],
        [1.9996, 2.0001],
        [2.0001, 2.0007]], requires_grad=True)
Parameter containing:
tensor([2.0002, 2.0003, 2.0014], requires_grad=True)
Parameter containing:
tensor([[1.9994, 1.9998, 2.0004]], requires_grad=True)
Parameter containing:
tenso